In [83]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
import re

In [24]:
pd.set_option('display.max_rows',None)
pd.set_option('display.max_colwidth', None)

In [31]:
options = Options() 
driver = webdriver.Chrome(options=options)
driver.get('https://www.clasificadosonline.com/RealEstate.asp')

In [49]:
multi_check_box_toggle = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[1]')
multi_check_box_toggle.click()
time.sleep(2)

In [50]:
select_all = driver.find_element(By.XPATH,'//*[@id="form1"]/div/table[2]/tbody/tr/td/div[2]/div[1]/input')
select_all.click()
time.sleep(2)

In [51]:
time.sleep(10)
ver_listado = driver.find_element(By.XPATH,'//*[@id="BtnSearchListing"]')
ver_listado.click()

In [54]:
order_by = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]')

In [ ]:
order_by = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]')
order_by.click()
ultimos_publicados = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]/option[7]')
ultimos_publicados.click()

In [12]:
list_cards = []

for i in range(1,16):
    table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
    table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
    table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
    table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
    table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
    table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
    table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
    
    table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
    table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
    table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
    table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
    table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
    table_2_location = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
    table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
    
    dictionary_cards = {}
    dictionary_cards_2 = {}
    try:
        table_1_type_2 = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_1_type_2_text = driver.find_element(By.CSS_SELECTOR, table_1_type_2).text
        dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text + ' ' + table_1_type_2_text
        
        table_2_type_2 = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_2_type_2_text = driver.find_element(By.CSS_SELECTOR, table_2_type_2).text
        dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text + ' ' + table_2_type_2_text

    except NoSuchElementException:
        dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
        dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
    try:
        table_1_broker = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        dictionary_cards['Broker'] = driver.find_element(By.CSS_SELECTOR, table_1_broker).get_attribute('alt')

        table_2_broker = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        dictionary_cards_2['Broker'] = driver.find_element(By.CSS_SELECTOR, table_2_broker).get_attribute('alt')

    except NoSuchElementException:
        dictionary_cards['Broker'] = 'No Broker'
        dictionary_cards_2['Broker'] = 'No Broker'

    dictionary_cards['Title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
    dictionary_cards['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
    dictionary_cards['Price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
    dictionary_cards['Location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
    dictionary_cards['Link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
    list_cards.append(dictionary_cards)

    dictionary_cards_2['Title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
    dictionary_cards_2['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
    dictionary_cards_2['Price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
    dictionary_cards_2['Location'] = driver.find_element(By.CSS_SELECTOR, table_2_location).text
    dictionary_cards_2['Link'] = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')
    list_cards.append(dictionary_cards_2)


In [ ]:
df = pd.json_normalize(list_cards)
df

In [ ]:
next_page = driver.find_element(By.XPATH,'//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
next_page.click()

In [5]:
last_modified = driver.execute_script("return document.lastModified;")

In [6]:
last_modified

'02/26/2024 23:01:12'

In [14]:
import requests

# Send a GET request to the webpage
response = requests.get("https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4774562")  # Replace this URL with the URL of your webpage

# Check the Last-Modified header in the response
last_modified_header = response.headers.get("Last-Modified")

# Print the Last-Modified header
print("Last modified (from HTTP header):", last_modified_header)

Last modified (from HTTP header): None


In [11]:
all_cards = []

# Infinite loop to navigate through pages
for x in range(3):
    # Extract data from the current page
    list_cards = []

    for i in range(1, 16):  # Adjust the range as per the number of rows on a page
        dictionary_cards = {}
        dictionary_cards_2 = {}
        
        # Table 1 Data
        table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
        table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'

        try:
            dictionary_cards['Title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
            dictionary_cards['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
            dictionary_cards['Price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
            dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
            dictionary_cards['Location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
            dictionary_cards['Link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
            list_cards.append(dictionary_cards)
        except NoSuchElementException:
            pass

        # Table 2 Data (similar logic for table_2)
        table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
        table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_2_location = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'

        try:
            dictionary_cards_2['Title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
            dictionary_cards_2['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
            dictionary_cards_2['Price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
            dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
            dictionary_cards_2['Location'] = driver.find_element(By.CSS_SELECTOR, table_2_location).text
            dictionary_cards_2['Link'] = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')
            list_cards.append(dictionary_cards_2)
        except NoSuchElementException:
            pass

    # Append the current page's data to the main list
    all_cards.extend(list_cards)

    # Try to click the "Next" button
    try:
        time.sleep(10)
        next_page = driver.find_element(By.XPATH, '//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
        next_page.click()
        time.sleep(7)  # Add delay to allow page to load
    except NoSuchElementException:
        print("No more pages available.")
        break

# Create a single DataFrame from all collected data
df = pd.DataFrame(all_cards)

# Save to CSV or process further
df.to_csv("classifieds_data.csv", index=False)

print("Data collection completed.")

Data collection completed.


In [90]:
def clean_real_estate_data(df):
    # Clean the 'Type' column by removing leading commas and whitespace
    df['Type'] = df['Type'].str.replace(r'^,\s*', '', regex=True)

    # Clean the 'Broker' column using the combined logic
    def clean_broker(broker_name):
        try:
            # Remove any text starting with " - " followed by anything after it
            broker_name = re.sub(r'\s*-\s*.*$', '', broker_name)
            
            # Remove any text starting with "#" followed by digits
            broker_name = re.sub(r'\s*#\d+', '', broker_name)
            
            # Remove any text starting with "lic." followed by digits
            broker_name = re.sub(r'\s*lic\.\s*\d+', '', broker_name)
            
        except Exception as e:
            print(f"An error occurred: {e}")

        # Return the cleaned broker name
        return broker_name.strip()
    
    # Apply the clean_broker function to the 'Broker' column
    df['Broker'] = df['Broker'].apply(clean_broker)

    # Drop the 'Title' column if needed
    # df.drop(columns=['Title'], inplace=True)

    # Extract the number of bedrooms and bathrooms from the 'Rooms' column
    df[['Bedrooms', 'Bathrooms']] = df['Rooms'].str.extract(r'(\d+)\s+Cuartos.*\|\s*(\d+)\s+Baños', expand=True)
    df[['Bedrooms', 'Bathrooms']] = df[['Bedrooms', 'Bathrooms']].fillna(0)

    df['Location'] = df['Location'].fillna('')

    # Split the 'Location' column by the comma into two columns
    df[['Neighborhood', 'City']] = df['Location'].str.split(',', n=1, expand=True)

    # Strip any leading or trailing whitespace from the new columns
    df['Neighborhood'] = df['Neighborhood'].str.strip()
    df['City'] = df['City'].str.strip()

    df['Broker'] = df['Broker'].str.replace('ClasificadosOnline', '').str.strip()

    return df

In [88]:
# Initialize an empty list to store all data
all_cards = []

# Initialize a page counter (optional, for debugging or tracking pages)
page_number = 1

# Infinite loop to navigate through pages
for x in range(3):

    dropdown_element = driver.find_element(By.XPATH, '//*[@id="jumpMenu"]')
    select = Select(dropdown_element)
    selected_option = select.first_selected_option

    print(f"Scraping page {page_number}...")

    if selected_option.text !=  'Ultimos Publicados (Más Recientes Primeros)':
        dropdown_element.click()
        time.sleep(2)
        ultimos_publicados = driver.find_element(By.XPATH,'//*[@id="jumpMenu"]/option[7]')
        ultimos_publicados.click()
        time.sleep(7)


    # Extract data from the current page
    list_cards = []

    for i in range(1, 16):  # Adjust the range as per the number of rows on a page
        dictionary_cards = {}
        dictionary_cards_2 = {}
        
        # Table 1 Data
        table_1 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(1)'
        table_1_title = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_1_rooms = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_1_price = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_1_type = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_1_type_2 = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_1_location = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_1_link = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
        table_1_broker = table_1 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        
        try:
            dictionary_cards['Title'] = driver.find_element(By.CSS_SELECTOR, table_1_title).text
            dictionary_cards['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_1_rooms).text
            dictionary_cards['Price'] = driver.find_element(By.CSS_SELECTOR, table_1_price).text
            
            # type validation, since a property can have two types
            try: 

                table_1_type_2_text = driver.find_element(By.CSS_SELECTOR, table_1_type_2).text
                dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text + ' ' + table_1_type_2_text

            except NoSuchElementException:
                dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text


            dictionary_cards['Type'] = driver.find_element(By.CSS_SELECTOR, table_1_type).text
            dictionary_cards['Location'] = driver.find_element(By.CSS_SELECTOR, table_1_location).text
            dictionary_cards['Link'] = driver.find_element(By.CSS_SELECTOR, table_1_link).get_attribute('href')
            
            if "MultipleSellers" in dictionary_cards['Link']:
                dictionary_cards['Broker'] = 'Multiple Sellers'

            else:

                # not all properties have brokers
                try:
                    dictionary_cards['Broker'] = driver.find_element(By.CSS_SELECTOR, table_1_broker).get_attribute('alt')
                except NoSuchElementException:
                    dictionary_cards['Broker'] = 'No Broker'

         
            list_cards.append(dictionary_cards)
            
        except NoSuchElementException:
            pass

        # Table 2 Data (similar logic for table_2)
        table_2 = '#listing > table > tbody > tr > td > table > tbody > tr:nth-child(2) > td > div > div:nth-child(2) > table:nth-child(3)'
        table_2_title = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a > div > span'
        table_2_rooms = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(1) > span'
        table_2_price = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(2) > font'
        table_2_type = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(4)'
        table_2_type_2 = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(1) > div:nth-child(2) > span:nth-child(5)'
        table_2_location = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(3) > td'
        table_2_link = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(1) > td > a'
        table_2_broker = table_2 + f' > tbody > tr:nth-child({i}) > td > div.dv-classified-row.dv-classified-row-v2 > table > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td:nth-child(2) > center > a > img'
        
        try:
            dictionary_cards_2['Title'] = driver.find_element(By.CSS_SELECTOR, table_2_title).text
            dictionary_cards_2['Rooms'] = driver.find_element(By.CSS_SELECTOR, table_2_rooms).text
            dictionary_cards_2['Price'] = driver.find_element(By.CSS_SELECTOR, table_2_price).text
            
            # type validation, since a property can have two types
            try:
                table_2_type_2_text = driver.find_element(By.CSS_SELECTOR, table_2_type_2).text
                dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text + ' ' + table_2_type_2_text
            except NoSuchElementException:
                dictionary_cards_2['Type'] = driver.find_element(By.CSS_SELECTOR, table_2_type).text
            
            dictionary_cards_2['Location'] = driver.find_element(By.CSS_SELECTOR, table_2_location).text
            dictionary_cards_2['Link'] = driver.find_element(By.CSS_SELECTOR, table_2_link).get_attribute('href')

            if "MultipleSellers" in dictionary_cards_2['Link']:
                dictionary_cards_2['Broker'] = 'Multiple Sellers'

            else:
                try:
                    dictionary_cards_2['Broker'] = driver.find_element(By.CSS_SELECTOR, table_2_broker).get_attribute('alt')
                except NoSuchElementException:
                    dictionary_cards_2['Broker'] = 'No Broker'
    
            list_cards.append(dictionary_cards_2)
            
        except NoSuchElementException:
            pass

    # Append the current page's data to the main list
    all_cards.extend(list_cards)

    # Try to click the "Next" button
    try:
        time.sleep(10)
        next_page = driver.find_element(By.XPATH, '//*[@id="listing"]/table/tbody/tr/td/table/tbody/tr[2]/td/table[5]/tbody/tr[1]/td[3]/div/a')
        next_page.click()
        time.sleep(7)  # Add delay to allow page to load
        page_number += 1
    except NoSuchElementException:
        print("No more pages available.")
        break

# Create a single DataFrame from all collected data
df = pd.DataFrame(all_cards)
cleaned_df = clean_real_estate_data(df)

Scraping page 1...
Scraping page 2...
Scraping page 3...


In [92]:
cleaned_df

,Title,Rooms,Price,Type,Location,Link,Broker,Bedrooms,Bathrooms,Neighborhood,City
0,Lot 5 Sr 787 Km 4.4,3 Cuartos | 2 Baños,"$201,500",Casa,"Barrio-Certenejas , Cidra",https://www.clasificadosonline.com/REFSMultipleSellers.asp?ID=4837237,Multiple Sellers,3,2,Barrio-Certenejas,Cidra
1,RIVERSIDE BAR @ BO. RIO ARRIBA FAJARD...,1 Cuartos | 4 Baños,"$268,000",Comercial,"Barrio-Rio Arriba , Fajardo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4802028,Rio Arriba de ENTER Real Estate Network,1,4,Barrio-Rio Arriba,Fajardo
2,718 Luis Armanza St,3 Cuartos | 2 Baños,"$177,000",Casa,"Urbanizacion-Fair View , San Juan - Río Piedras",https://www.clasificadosonline.com/REFSMultipleSellers.asp?ID=4837238,Multiple Sellers,3,2,Urbanizacion-Fair View,San Juan - Río Piedras
3,Singular Property in prime Santurce c...,,"$2,500,000",Comercial,"Barrio-Santurce , San Juan - Santurce",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4810398,Santurce de ALONSO,0,0,Barrio-Santurce,San Juan - Santurce
4,Af-10 Borgona St,3 Cuartos | 2 Baños,"$95,400",Casa,"Urbanizacion-San Jose , San Juan - Río Piedras",https://www.clasificadosonline.com/REFSMultipleSellers.asp?ID=4816106,Multiple Sellers,3,2,Urbanizacion-San Jose,San Juan - Río Piedras
5,BEACH FRONT | RARE FIND | COSTA AZUL,4 Cuartos | 2 Baños,"$850,000",Casa,"Urbanizacion-Costa Azul , Luquillo",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4790783,Costa Azul de GLORIBEL VEGA,4,2,Urbanizacion-Costa Azul,Luquillo
6,A14 Yagrumo St,4 Cuartos | 3 Baños,"$485,000",Casa,"Urbanizacion-Villa Lucia , Arecibo",https://www.clasificadosonline.com/REFSMultipleSellers.asp?ID=4821878,Multiple Sellers,4,3,Urbanizacion-Villa Lucia,Arecibo
7,MULTIFAMILIAR - CONTADORES SEPARADOS,5 Cuartos | 3 Baños,"$209,000",MultiFamiliar,"Sector-La Ponderosa , Vega Alta",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4814965,La Ponderosa de NAUTIC REAL ESTATE,5,3,Sector-La Ponderosa,Vega Alta
8,97 7 Street Sr 973,3 Cuartos | 3 Baños,"$90,000",Casa,"Barrio-Mariana , Naguabo",https://www.clasificadosonline.com/REFSMultipleSellers.asp?ID=4814717,Multiple Sellers,3,3,Barrio-Mariana,Naguabo
9,"Bo. Furnias, Las Marías",,"$150,000",Finca,"Barrio-Furnias , Las Marías",https://www.clasificadosonline.com/UDRealEstateDetail.asp?ID=4835868,Furnias de Real Home of America,0,0,Barrio-Furnias,Las Marías


In [93]:
cleaned_df.to_csv("classifieds_data.csv", index=False)